In [1]:
import numpy as np

In [2]:
X = 1500
Y = 1500
Z = 20
K = 3
data = np.random.rand(X,Y,Z)

# Try 1

In [3]:
"""
def vectors_comparison(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2))
    return res

%%prun -l 20
comp_res = {}
for x in range(X): 
    comp_res[x] = {}
    
    for y in range(Y):
        comp_res[x][y] = {}
        col = data[x, y, :]
        
        for x_iter in range(max(0, x-K), min(X, x+K+1)):  #  for x_iter in range(x, min(X, x+K+1)): 
            comp_res[x][y][x_iter] = {}
            
            for y_iter in range(max(0, y-K), min(Y, y+K+1)):

                if x_iter in comp_res: # придумать попроще
                    if y_iter in comp_res[x_iter]:
                        if x in comp_res[x_iter][y_iter]:
                            if y in comp_res[x_iter][y_iter][x]:
                                comp_res[x][y][x_iter][y_iter] = comp_res[x_iter][y_iter][x][y]
                                continue

                if (x_iter != x) or (y_iter != y): # подумать над форматом кэширования
                    comp_res[x][y][x_iter][y_iter] = vectors_comparison(col, data[x_iter, y_iter])

neighbours_comp = np.zeros((X,Y)).reshape(X,Y)
for x in range(X):
    for y in range(Y):
        tmp_arr = []
        for x_iter in comp_res[x][y]:
            for y_iter in comp_res[x][y][x_iter]:
                tmp_arr.append(comp_res[x][y][x_iter][y_iter])
        neighbours_comp[x][y] = np.mean(tmp_arr)
"""

'\ndef vectors_comparison(vector1, vector2):\n    res = np.multiply(vector1, vector2)\n    res = np.sum(np.power(res, 2))\n    return res\n\n%%prun -l 20\ncomp_res = {}\nfor x in range(X): \n    comp_res[x] = {}\n    \n    for y in range(Y):\n        comp_res[x][y] = {}\n        col = data[x, y, :]\n        \n        for x_iter in range(max(0, x-K), min(X, x+K+1)):  #  for x_iter in range(x, min(X, x+K+1)): \n            comp_res[x][y][x_iter] = {}\n            \n            for y_iter in range(max(0, y-K), min(Y, y+K+1)):\n\n                if x_iter in comp_res: # придумать попроще\n                    if y_iter in comp_res[x_iter]:\n                        if x in comp_res[x_iter][y_iter]:\n                            if y in comp_res[x_iter][y_iter][x]:\n                                comp_res[x][y][x_iter][y_iter] = comp_res[x_iter][y_iter][x][y]\n                                continue\n\n                if (x_iter != x) or (y_iter != y): # подумать над форматом кэширования\n  

# Try 2

In [4]:
%%time

"""
res = np.zeros(data.shape[0:2])
for x in range(X):
    for y in range(Y):        
        col = data[x, y, :]        
        for x_idx in range(-K, K+1):
            for y_idx in range(-K, K+1):
                if (0 <= x + x_idx < X) and (0 <= y + y_idx < Y):
                    res[x, y] += vectors_comparison(col, data[x + x_idx, y + y_idx])
        res[x, y] /= (min(x + x_idx, X - 1) - max(0, x - x_idx) + 1) * (min(y + y_idx, Y - 1) - max(0, y - y_idx) + 1) - 1
        
"""

CPU times: user 1e+03 ns, sys: 3 µs, total: 4 µs
Wall time: 8.58 µs


'\nres = np.zeros(data.shape[0:2])\nfor x in range(X):\n    for y in range(Y):        \n        col = data[x, y, :]        \n        for x_idx in range(-K, K+1):\n            for y_idx in range(-K, K+1):\n                if (0 <= x + x_idx < X) and (0 <= y + y_idx < Y):\n                    res[x, y] += vectors_comparison(col, data[x + x_idx, y + y_idx])\n        res[x, y] /= (min(x + x_idx, X - 1) - max(0, x - x_idx) + 1) * (min(y + y_idx, Y - 1) - max(0, y - y_idx) + 1) - 1\n        \n'

# Try 3: numba

In [5]:
import numba

In [6]:
from numba import njit

In [7]:
X = np.int32(1500)
Y = np.int32(1500)
Z = np.int32(20)
K = np.int32(3)
data = np.random.rand(X,Y,Z).astype('float32')

In [8]:
%%time

@njit(numba.types.float32(numba.types.float32[:], 
                          numba.types.float32[:]),
      fastmath=True)
def vectors_comparison(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2))
    return res

@njit(numba.types.float32[:, :](numba.types.float32[:, :, :], 
                                numba.types.int32, 
                                numba.types.int32, 
                                numba.types.int32, 
                                numba.types.int32),
    locals={'res': numba.types.float32[:, :],
            'col': numba.types.float32[:]}, parallel=True)
def get_comparison(data, X, Y, Z, K):
    res = np.zeros(data.shape[0:2], dtype='float32')
    for x in range(X):
        for y in range(Y):        
            col = data[x, y, :]        
            for x_idx in range(-K, K+1):
                for y_idx in range(-K, K+1):
                    sss = np.float32(0.0)
                    if (0 <= x + x_idx < X) and (0 <= y + y_idx < Y):
                        sss += vectors_comparison(col, data[x + x_idx, y + y_idx])
            count = (min(x + x_idx, X - 1) - max(0, x - x_idx) + 1) * (min(y + y_idx, Y - 1) - max(0, y - y_idx) + 1) - 1
            res[x, y] = sss / count
    return res

CPU times: user 1.92 s, sys: 6.06 s, total: 7.98 s
Wall time: 1.22 s


In [9]:
%%time
res = get_comparison(data, X, Y, Z, K)

CPU times: user 16.8 s, sys: 188 ms, total: 17 s
Wall time: 16.7 s


# Try 4: numpy

In [10]:
def vectors_comparison(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2), axis = 2)
    return res

def compare_matrices(a_pad, xk, yk, K):
    comparison = vectors_comparison(a_pad[K:-K,K:-K,:], a_pad[xk+K:xk+a_pad.shape[0]-K, yk+K:yk+a_pad.shape[1]-K,:])
    res = comparison + np.pad(comparison, K)[-xk+K:-xk+a_pad.shape[1]-K, -yk+K:-yk+a_pad.shape[0]-K]
    return res

def get_comparison(data, X, Y, Z, K):
    K = K+1
    a_pad = np.pad(data, K)
    result = np.zeros(data.shape[:2])
    for xk in range(0, K):
        for yk in range(0, K):
            if (xk!=0) or (yk!=0):   
                result += compare_matrices(a_pad, xk, yk, K)
                if (yk!=0) and (xk!=0):
                    result += compare_matrices(a_pad, xk, -yk, K)     
    return result

In [11]:
%%time
res = get_comparison(data, X, Y, Z, K)

CPU times: user 16.6 s, sys: 2.33 s, total: 19 s
Wall time: 19 s


In [12]:
def vectors_comparison(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2), axis = 2)
    return res

def compare_matrices(a_pad, data, xk, yk, K):
    comparison = vectors_comparison(data, a_pad[xk:xk+data.shape[0], K+yk:K+yk+data.shape[1], :])
    res = comparison + np.pad(comparison, ((xk, 0), (K, K)))[0:data.shape[0], K-yk:K-yk+data.shape[1]]
    return res

def get_comparison(data, X, Y, Z, K):
    K = K + 1
    a_pad = np.pad(data, ((0, K), (K, K), (0, 0)))
    result = np.zeros(data.shape[:2])
    for xk in range(0, K):
        for yk in range(0, K):
            if (xk!=0) or (yk!=0):   
                result += compare_matrices(a_pad, data, xk, yk, K)
                if (yk!=0) and (xk!=0):
                    result += compare_matrices(a_pad, data, xk, -yk, K)            
    return result

In [13]:
%%time
res2 = get_comparison(data, X, Y, Z, K)

CPU times: user 13.2 s, sys: 1.62 s, total: 14.8 s
Wall time: 14.8 s


In [14]:
np.allclose(res, res2)

True

In [15]:
%%prun -l 20
res2 = get_comparison(data, X, Y, Z, K)

         1775 function calls (1700 primitive calls) in 14.380 seconds

   Ordered by: internal time
   List reduced from 53 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       24   12.920    0.538   14.164    0.590 <ipython-input-12-e937474291bf>:1(vectors_comparison)
       49    1.243    0.025    1.243    0.025 {method 'reduce' of 'numpy.ufunc' objects}
       25    0.091    0.004    0.091    0.004 arraypad.py:86(_pad_simple)
        1    0.064    0.064   14.378   14.378 <ipython-input-12-e937474291bf>:11(get_comparison)
       24    0.052    0.002   14.265    0.594 <ipython-input-12-e937474291bf>:6(compare_matrices)
       51    0.002    0.000    0.002    0.000 arraypad.py:129(_set_pad_area)
        1    0.001    0.001   14.380   14.380 <string>:1(<module>)
        1    0.001    0.001    0.001    0.001 {built-in method numpy.zeros}
       25    0.001    0.000    0.097    0.004 arraypad.py:532(pad)
      125    0.001    0.000 